```
memory
```

In [ ]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
memory.load_memory_variables({})

In [ ]:
memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

In [ ]:
# 返回message格式
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
memory.load_memory_variables({})


In [ ]:
memory.chat_memory.messages

In [ ]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")
history.messages

In [3]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature=0)
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""


prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("你是谁!")
memory.chat_memory.add_ai_message("我是一个诗人")



In [ ]:
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation({"question": "你是诗人吗"})


In [ ]:
memory.load_memory_variables({})

---------------

In [1]:
from langchain.memory import ConversationKGMemory
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})
memory.load_memory_variables({"input": "who is sam"})

{'history': 'On sam: sam is a friend.'}

In [4]:
# 在chain中使用kg
from langchain.chains import ConversationChain
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation_with_kg = ConversationChain(
    llm=llm, verbose=True, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

In [5]:
conversation_with_kg.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi, what's up?
AI:

> Finished chain.


' Hello! I am an AI programmed to assist with various tasks and provide information. Currently, I am running on a server with 64 GB of RAM and a 2.5 GHz processor. I am also connected to the internet with a speed of 100 Mbps. Is there something specific you would like to know?'

In [6]:
conversation_with_kg.predict(
    input="My name is James and I'm helping Will. He's an engineer."
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My name is James and I'm helping Will. He's an engineer.
AI:

> Finished chain.


" Hello James, it's nice to meet you. I am an AI designed to assist with various tasks. Will sounds like a very interesting person. As an engineer, he must have a strong understanding of math, science, and problem-solving. Is there anything specific you need help with?"

In [7]:
conversation_with_kg.predict(input="What do you know about Will?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Will: Will is an engineer.

Conversation:
Human: What do you know about Will?
AI:

> Finished chain.


' Will is an engineer. He graduated from the University of California, Berkeley with a degree in mechanical engineering. He currently works at a tech company in Silicon Valley and specializes in designing and building robots for industrial use. He is also an avid hiker and enjoys spending his weekends exploring the nearby mountains and trails. Is there anything specific you would like to know about Will?'

In [9]:
print(conversation_with_kg.memory.kg.get_triples())

[('James', 'Will', 'is helping'), ('Will', 'engineer', 'is an'), ('Will', 'tech company', 'works at'), ('Will', 'designing and building robots', 'specializes in'), ('Will', 'hiker', 'is an avid'), ('Will', 'weekends exploring mountains and trails', 'enjoys spending')]


In [10]:

from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

{'history': "\nThe human greets the AI and asks what's up. The AI responds by asking what's going on."}

In [11]:
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)


In [12]:
response = conversation_with_summary("我喜欢运动，所以我想买一个篮球")
print(response)

/opt/homebrew/anaconda3/envs/pytorch/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 我喜欢运动，所以我想买一个篮球
AI:

> Finished chain.
{'input': '我喜欢运动，所以我想买一个篮球', 'history': '', 'response': ' 那你是一个运动爱好者啊！我也很喜欢运动，尤其是篮球。你想买什么样的篮球呢？有很多不同的品牌和款式，我可以帮你推荐一些。你喜欢什么样的颜色和材质？还有你的预算是多少？我可以帮你找到最适合你的篮球。'}


In [13]:
response = conversation_with_summary("我喜欢威尔逊品牌的")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human expresses their interest in sports and wanting to purchase a basketball. The AI responds by saying it is also a sports enthusiast and can help recommend different brands and styles based on the human's preferences and budget.
Human: 我喜欢威尔逊品牌的
AI:

> Finished chain.
{'input': '我喜欢威尔逊品牌的', 'history': "\nThe human expresses their interest in sports and wanting to purchase a basketball. The AI responds by saying it is also a sports enthusiast and can help recommend different brands and styles based on the human's preferences and budget.", 'response': ' 威尔逊是一家非常知名的体育用品品牌，专门生产篮球、网球和高尔夫球等运动用具。它的篮球产品质量优良，深受运动员和球迷的喜爱。您可以选择威尔逊的室内篮球，它具有良好的弹性和控球性能，非常适合在室内场地使用。另外，威尔逊还有许多

In [14]:
response = conversation_with_summary("我又来了，有什么篮球推荐吗")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human expresses their interest in sports and wanting to purchase a basketball. The AI responds by saying it is also a sports enthusiast and can help recommend different brands and styles based on the human's preferences and budget. The AI suggests the Wilson brand, known for its high-quality basketball products and popularity among athletes and fans. They recommend the indoor basketball for its good bounce and ball control, and mention that there are various styles and prices to choose from.
Human: 我又来了，有什么篮球推荐吗
AI:

> Finished chain.
{'input': '我又来了，有什么篮球推荐吗', 'history': "\nThe human expresses their interest in sports and wanting to purchase a basketball. The AI 